In [1]:
pip install lxml

Note: you may need to restart the kernel to use updated packages.


In [2]:
import requests
import pandas as pd
import lxml
import numpy as np

## Reading in the first table from URL

In [19]:
url= 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
results= requests.get(url)
df= pd.read_html(results.text)

- How many tables does our list contain?

In [20]:
len(df)

3

- Clearly, we need to deal with only the following table:

In [21]:
df[0]

,Postal code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
...,...,...,...
175,M5Z,Not assigned,NaN
176,M6Z,Not assigned,NaN
177,M7Z,Not assigned,NaN
178,M8Z,Etobicoke,Mimico NW / The Queensway West / South of Bloo...


In [22]:
df= df[0]
print('Shape of our dataframe is', df.shape)
print('Number of NA values present in our df is', df.isna().sum().sum())

Shape of our dataframe is (180, 3)
Number of NA values present in our df is 77


## Carrying out step #3 in assignment:

- The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood

In [23]:
df.columns

Index(['Postal code', 'Borough', 'Neighborhood'], dtype='object')

- Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [26]:
df= df[df['Borough']!='Not assigned']

In [27]:
print('Shape of our dataframe is', df.shape)
print('Number of NA values present in our df is', df.isna().sum().sum())

Shape of our dataframe is (103, 3)
Number of NA values present in our df is 0


- If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.
    - Our transformed dataframe does not contain any NA values in nay columns.
    - However it might contain 'Not assigned' value in 'Neighborhood' column, we should check for that.

In [29]:
'Not assigned' in df['Neighborhood'].unique().tolist()

False

- More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.
    - Seems wikipedia page has been updated to reflect what was required in the assignment.
    - We are simply replacing the '/' by a ','

In [30]:
df.head()

,Postal code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
5,M6A,North York,Lawrence Manor / Lawrence Heights
6,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


In [34]:
df['Neighborhood']= df['Neighborhood'].str.replace('/', ',')
df.head()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,Postal code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park , Harbourfront"
5,M6A,North York,"Lawrence Manor , Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government"


- Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.
    - No assumptions were made, markdown cells were added, notebook cleaned.

- In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

In [35]:
print('Number of rows in our dataframe:', df.shape[0])

Number of rows in our dataframe: 103


## ================================= PART 1 complete =================================

## Getting the latitude and the longitude coordinates of each neighborhood:

- Since the Geocoder package is said to be unreliable, we shall be using the csv file @ https://cocl.us/Geospatial_data.

- Use the Geocoder package or the csv file to create the following dataframe:

In [41]:
lldf= pd.read_csv('https://cocl.us/Geospatial_data')
lldf.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [42]:
lldf.shape

(103, 3)

- Both our dataframes contain equal number of rows. We plan on joining both on the 'Postal Code' column.
- We cannot be sure whether that column is identical or not in both the dataframes.
    - We can convert that column from both datarames into sets & check for equality.

In [43]:
set(df['Postal code']) == set(lldf['Postal Code'])

True

- Now we can proceed with our join:

In [51]:
df= pd.merge(df, lldf, left_on= ['Postal code'], right_on= ['Postal Code']).drop(['Postal Code'], axis= 1)

In [52]:
df

,Postal code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor , Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway , Montgomery Road , Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,Business reply mail Processing CentrE,43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South , King's Mill Park , Sunnylea ,...",43.636258,-79.498509


## ================================= PART 2 complete =================================